In [224]:
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [225]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [226]:
url= 'https://api.foursquare.com/v2/venues/search'
NEAR='Providence,RI'
RADIUS='5000'
category = '4bf58dd8d48988d10f941735'

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, NEAR, VERSION, category, RADIUS)
            
        # make the GET request
indianresults = requests.get(url).json()

category = '4d4b7105d754a06374d81259'
limit='50'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&categoryId={}&intent=browse&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, NEAR, VERSION, category, RADIUS,limit)
foodresults  = requests.get(url).json()
NEAR='Providence_College,Providence,RI'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&categoryId={}&intent=browse&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, NEAR, VERSION, category, RADIUS,limit)
foodresults2  = requests.get(url).json()
ll = '41.7989376,-71.415065'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={}&v={}&categoryId={}&intent=browse&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ll, VERSION, category, RADIUS,limit)
foodresults3= requests.get(url).json()

In [227]:
indian= pd.DataFrame(indianresults)
food=pd.DataFrame(foodresults)
food2=pd.DataFrame(foodresults2)
food3=pd.DataFrame(foodresults3)

In [228]:
venues = indianresults['response']['venues']
foodvenues=foodresults['response']['venues']
foodvenues2=foodresults2['response']['venues']
foodvenues3=foodresults3['response']['venues']

In [229]:
dataframe0 = json_normalize(venues)
dataframe1= json_normalize(foodvenues)
dataframe2= json_normalize(foodvenues2)
dataframe3= json_normalize(foodvenues3)

In [230]:
for automate in range(4):
    filtered_columns='filtered_columns'+str(automate)
    dataframe_filtered='dataframe_filtered'+str(automate)
    if automate==0:
        df=dataframe0
    if automate==1:
        df=dataframe1
    if automate==2:
        df=dataframe2
    if automate==3:
        df=dataframe3
    categories_list='categories_list'+str(automate)
    dataframe_filtered='dataframe_filtered'+str(automate)
    dff='dff'+str(automate)

    globals()[filtered_columns] = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
    globals()[dataframe_filtered] = df.loc[:, globals()[filtered_columns]]


    def get_category_type(row):
        try:
            globals()[categories_list] = row['categories']
        except:
            globals()[categories_list] = row['venue.categories']
        
        if len(globals()[categories_list]) == 0:
            return None
        else:
            return globals()[categories_list][0]['name']


    globals()[dataframe_filtered]['categories'] = globals()[dataframe_filtered].apply(get_category_type, axis=1)


    globals()[dataframe_filtered].columns = [column.split('.')[-1] for column in globals()[dataframe_filtered].columns]
    globals()[dff]=globals()[dataframe_filtered]
dff0.drop([0,5,8,9],inplace=True)

In [231]:
frames = [dff1, dff2, dff3]
merge = pd.concat(frames,sort=False)
merge.shape

(150, 16)

In [232]:
address = 'Providence, RI'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [273]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13,)
locationlist2 = merge[["lat","lng"]].values.tolist()
labels2 = merge["name"].values.tolist()

for point in range(len(locationlist2)):
    popup = folium.Popup(labels2[point], parse_html=True)
    folium.CircleMarker(locationlist2[point], popup=popup,radius=5,color='red',fill=True,fill_color='red',fill_opacity=0.7).add_to(venues_map)


locationlist = dff0[["lat","lng"]].values.tolist()
labels = dff0["name"].values.tolist()

for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(venues_map)


venues_map

In [234]:
dfc=dff0.drop(['name','categories','address','cc','city','country','crossStreet','formattedAddress','labeledLatLngs','postalCode','state','id'],1)
dfc2=merge.drop(['name','categories','address','cc','city','country','crossStreet','formattedAddress','labeledLatLngs','postalCode','state','id'],1)

In [153]:
pip install shapely

     |████████████████████████████████| 1.5MB 17.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [250]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
for automate in range(2):
    if automate==0:
        coords = dfc.as_matrix(columns=['lat', 'lng'])
    if automate==1:
        coords = dfc2.as_matrix(columns=['lat', 'lng'])
    clusters='clusters'+str(automate)
    kms_per_radian = 6371.0088
    epsilon = 1.5 / kms_per_radian
    db = DBSCAN(eps=epsilon,min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    globals()[clusters] = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    print(clusters)
    print('Number of clusters: {}'.format(num_clusters))

clusters0
Number of clusters: 6
clusters1
Number of clusters: 5


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [260]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters0.map(get_centermost_point)

In [274]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)



for marker in range(6):
    popup=folium.Popup('Indian Restaurant Cluster Center')
    markers_colors = []
    lat=centermost_points[marker][0]
    long=centermost_points[marker][1]
    folium.CircleMarker([lat, long],radius=5,fill=True,fill_opacity=1,fill_color='Red',color='Red',popup=popup).add_to(map_clusters)

map_clusters

In [264]:
centermost_points2 = clusters1.map(get_centermost_point)

In [275]:

for marker in range(5):
    popup = folium.Popup('Busy Food Area Cluster Center')
    markers_colors = []
    lat=centermost_points2[marker][0]
    long=centermost_points2[marker][1]
    folium.CircleMarker([lat, long],radius=40,fill=True,fill_opacity=0.2,fill_color='Green',color='Green',label='Food Area',popup=popup).add_to(map_clusters)
map_clusters